In [ ]:
import pandas as pd
from os.path import join
from typing import List, Dict
from datetime import datetime
import pathlib
from binreader.binread import MySmspec

pd.set_option('display.width', 1000)
pd.set_option('display.max_columns',10)

def read_smspecs(folder: str) -> Dict[str, MySmspec]:
    '''data = daily or yearly'''
    dir = pathlib.Path(folder)
    # эталонная выборка
    res=dict()
    for file in dir.rglob('result.smspec'): 
        key=file.parts[-2]
        type = int(key[:3], 2)
        smspec = MySmspec(file)
        res[key]=smspec
    return res

In [ ]:
# получаем смспеки, работает около 2 минут. с переменой класса MySmspec удобно работать
# в переменой smspecs Dict[str, MySmspec] str==название папки
smspecs=read_smspecs('tnav_results')

In [ ]:
# работает невероятно долго, и получаются не очень красивые csv c огромнымм количеством колонок (WGNAMES.count()*KEYWORDS.count() штук)
import os 
dir='files/csv'
if not os.path.exists(dir):
    os.makedirs(dir) 
for key in list(smspecs.keys()):
    keywords=smspecs[key].get_all_keywords
    wgnames=smspecs[key].get_all_groups+smspecs[key].get_all_wells
    # wgnames=['FIELD']
    df=smspecs[key].get_main(keywords, wgnames)
    df.to_csv(f'{dir}/{key}.csv')

In [ ]:
# работает чуть быстрее, все тоже самое
import os 
dir='files/csv'
if not os.path.exists(dir):
    os.makedirs(dir) 
for key in list(smspecs.keys()):
    keywords=smspecs[key].get_all_keywords
    wgnames=smspecs[key].get_all_groups+smspecs[key].get_all_wells
    df=smspecs[key].get_data
    # wgnames=['FIELD']
    df = smspecs[key].get_data.loc[smspecs[key].get_data["KEYWORDS"].isin(keywords) & smspecs[key].get_data["WGNAMES"].isin(wgnames)]
    cols=(df['WGNAMES']+':'+df['KEYWORDS']+'('+df['UNITS']+')').to_list()
    df=df.iloc[:, 4:].T
    df.columns=cols
    df.index=smspecs[key].get_all_dates
    df.to_csv(f'{dir}/{key}.csv')